In [1]:
import torch
from super_gradients.training import models
from torchinfo import summary
import numpy as np
import math
from numpy import random
import cv2
from roboflow import Roboflow
from super_gradients.training import Trainer
from super_gradients.training.dataloaders.dataloaders import (
    coco_detection_yolo_format_train, coco_detection_yolo_format_val)
from super_gradients.training.losses import PPYoloELoss
from super_gradients.training.metrics import DetectionMetrics_050, DetectionMetrics_050_095
from super_gradients.training.models.detection_models.pp_yolo_e import PPYoloEPostPredictionCallback
import logging
#logging.getLogger("super_gradients").setLevel(logging.WARNING)


The console stream is logged into /home/zw2688/sg_logs/console.log


[2023-11-28 23:25:41] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it
[2023-11-28 23:25:41] WARNING - __init__.py - Failed to import pytorch_quantization
2023-11-28 23:25:44.116724: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-28 23:25:44.498054: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-28 23:25:44.498073: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-28 23:25:44.498099: E tensorflow/compiler/

In [2]:
# model = models.get("yolo_nas_s", pretrained_weights="coco").to('cuda')

# rf = Roboflow(api_key="nz2w1UFnyFrM7e73WOzh")
# project = rf.workspace("nyu-figsb").project("basketballdetection-cki6r")
# dataset = project.version(14).download("yolov5")
!nvidia-smi

In [3]:
LOCATION = "basketballDetection-21"
CLASSES = ['basketball', 'hoop', 'person']
CHECKPOINT_DIR = 'yolonas_checkpoints'
NUM_WORKERS = 12
BATCH_SIZE = 64
MAX_EPOCHS = 150
trainer = Trainer(experiment_name="bball_detect_v21", ckpt_root_dir=CHECKPOINT_DIR)


In [4]:
dataset_params = {
    'data_dir': LOCATION,
    'train_images_dir':'train/images',
    'train_labels_dir':'train/labels',
    'val_images_dir':'val/images',
    'val_labels_dir':'val/labels',
    'test_images_dir':'test/images',
    'test_labels_dir':'test/labels',
    'classes': CLASSES
}

train_data = coco_detection_yolo_format_train(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['train_images_dir'],
        'labels_dir': dataset_params['train_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': BATCH_SIZE,
        'num_workers': NUM_WORKERS
    }
)

val_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['val_images_dir'],
        'labels_dir': dataset_params['val_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': BATCH_SIZE,
        'num_workers': NUM_WORKERS
    }
)

test_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['test_images_dir'],
        'labels_dir': dataset_params['test_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': BATCH_SIZE,
        'num_workers': NUM_WORKERS
    }
)

Tue Nov 28 23:25:48 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Quadro RTX 8000                On  | 00000000:06:00.0 Off |                    0 |
| N/A   33C    P8              26W / 250W |      3MiB / 46080MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

[2023-11-28 23:25:48] INFO - detection_dataset.py - Dataset Initialization in progress. `cache_annotations=True` causes the process to take longer due to full dataset indexing.
Indexing dataset annotations: 100%|██████████| 2068/2068 [00:02<00:00, 988.98it/s] 
This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
[2023-11-28 23:25:50] INFO - detection_dataset.py - Dataset Initialization in progress. `cache_annotations=True` causes the process to take longer due to full dataset indexing.
Indexing dataset annotations: 100%|██████████| 591/591 [00:00<00:00, 2077.02it/s]
[2023-11-28 23:25:50] INFO - detection_dataset.py - Dataset Initialization in progress. `cache_annotations=True` causes the proces

In [5]:
train_data.dataset.transforms.pop(2)

DetectionMixup('additional_samples_count': 1, 'non_empty_targets': True, 'input_dim': (640, 640), 'mixup_scale': [0.5, 1.5], 'prob': 1.0, 'enable_mixup': True, 'flip_prob': 0.5, 'border_value': 114)

In [6]:
model = models.get("yolo_nas_s", num_classes = 3, pretrained_weights = "coco")

[2023-11-28 23:25:51] INFO - checkpoint_utils.py - License Notification: YOLO-NAS pre-trained weights are subjected to the specific license terms and conditions detailed in 
https://github.com/Deci-AI/super-gradients/blob/master/LICENSE.YOLONAS.md
By downloading the pre-trained weight files you agree to comply with these terms.
[2023-11-28 23:25:51] INFO - checkpoint_utils.py - Successfully loaded pretrained weights for architecture yolo_nas_s


In [7]:
train_params = {
    'silent_mode': False,
    "MultiGPUMode": "DP",
    "average_best_models":True,
    "warmup_mode": "LinearEpochLRWarmup",
    "warmup_initial_lr": 1e-6,
    "lr_warmup_epochs": 3,
    "initial_lr": 1e-4,
    "lr_mode": "cosine",
    "cosine_final_lr_ratio": 0.1,
    "optimizer": "Adam",
    "optimizer_params": {"weight_decay": 0.0001},
    "zero_weight_decay_on_bias_and_bn": True,
    "ema": True,
    "ema_params": {"decay": 0.9, "decay_type": "threshold"},
    "max_epochs": MAX_EPOCHS,
    "mixed_precision": True,
    "loss": PPYoloELoss(
        use_static_assigner=False,
        num_classes=len(dataset_params['classes']),
        reg_max=16
    ),
    "valid_metrics_list": [
        DetectionMetrics_050(
            score_thres=0.1,
            top_k_predictions=300,
            num_cls=len(dataset_params['classes']),
            normalize_targets=True,
            post_prediction_callback=PPYoloEPostPredictionCallback(
                score_threshold=0.01,
                nms_top_k=1000,
                max_predictions=300,
                nms_threshold=0.7
            )
        ),
        DetectionMetrics_050_095(
            score_thres=0.1,
            top_k_predictions=300,
            num_cls=len(dataset_params['classes']),
            normalize_targets=True,
            post_prediction_callback=PPYoloEPostPredictionCallback(
                score_threshold=0.01,
                nms_top_k=1000,
                max_predictions=300,
                nms_threshold=0.7
            )
        )
    ],
    "metric_to_watch": 'mAP@0.50:0.95'
}

In [8]:
trainer.train(
    model=model,
    training_params=train_params,
    train_loader=train_data,
    valid_loader=val_data
)

[2023-11-28 23:25:51] INFO - sg_trainer.py - Starting a new run with `run_id=RUN_20231128_232551_738598`
[2023-11-28 23:25:51] INFO - sg_trainer.py - Checkpoints directory: yolonas_checkpoints/bball_detect_v21/RUN_20231128_232551_738598
[2023-11-28 23:25:51] INFO - sg_trainer.py - Using EMA with params {'decay': 0.9, 'decay_type': 'threshold'}


The console stream is now moved to yolonas_checkpoints/bball_detect_v21/RUN_20231128_232551_738598/console_Nov28_23_25_51.txt


[2023-11-28 23:26:25] INFO - sg_trainer_utils.py - TRAINING PARAMETERS:
    - Mode:                         Single GPU
    - Number of GPUs:               1          (2 available on the machine)
    - Full dataset size:            2067       (len(train_set))
    - Batch size per GPU:           64         (batch_size)
    - Batch Accumulate:             1          (batch_accumulate)
    - Total batch size:             64         (num_gpus * batch_size)
    - Effective Batch size:         64         (num_gpus * batch_size * batch_accumulate)
    - Iterations per epoch:         32         (len(train_loader))
    - Gradient updates per epoch:   32         (len(train_loader) / batch_accumulate)

[2023-11-28 23:26:25] INFO - sg_trainer.py - Started training for 150 epochs (0/149)

Validating: 100%|██████████| 10/10 [00:13<00:00,  1.30s/it]
[2023-11-28 23:27:49] INFO - base_sg_logger.py - Checkpoint saved in yolonas_checkpoints/bball_detect_v21/RUN_20231128_232551_738598/ckpt_best.pth
[2023-1

SUMMARY OF EPOCH 0
├── Train
│   ├── Ppyoloeloss/loss_cls = 2.8304
│   ├── Ppyoloeloss/loss_iou = 0.2709
│   ├── Ppyoloeloss/loss_dfl = 1.2127
│   └── Ppyoloeloss/loss = 4.1139
└── Validation
    ├── Ppyoloeloss/loss_cls = 2.4175
    ├── Ppyoloeloss/loss_iou = 0.2036
    ├── Ppyoloeloss/loss_dfl = 0.9464
    ├── Ppyoloeloss/loss = 3.3997
    ├── Precision@0.50 = 0.032
    ├── Recall@0.50 = 0.0004
    ├── Map@0.50 = 0.0037
    ├── F1@0.50 = 0.0003
    ├── Precision@0.50:0.95 = 0.0199
    ├── Recall@0.50:0.95 = 0.0
    ├── Map@0.50:0.95 = 0.0017
    └── F1@0.50:0.95 = 0.0



Validating epoch 1: 100%|██████████| 10/10 [00:14<00:00,  1.41s/it]
[2023-11-28 23:29:06] INFO - base_sg_logger.py - Checkpoint saved in yolonas_checkpoints/bball_detect_v21/RUN_20231128_232551_738598/ckpt_best.pth
[2023-11-28 23:29:06] INFO - sg_trainer.py - Best checkpoint overriden: validation mAP@0.50:0.95: 0.09839614480733871


...

Validating epoch 20: 100%|██████████| 10/10 [00:14<00:00,  1.50s/it]


SUMMARY OF EPOCH 20
├── Train
│   ├── Ppyoloeloss/loss_cls = 0.721
│   │   ├── Epoch N-1      = 0.7191 (↗ 0.0019)
│   │   └── Best until now = 0.7191 (↗ 0.0019)
│   ├── Ppyoloeloss/loss_iou = 0.1601
│   │   ├── Epoch N-1      = 0.1597 (↗ 0.0003)
│   │   └── Best until now = 0.1597 (↗ 0.0003)
│   ├── Ppyoloeloss/loss_dfl = 0.7876
│   │   ├── Epoch N-1      = 0.7832 (↗ 0.0045)
│   │   └── Best until now = 0.7825 (↗ 0.0051)
│   └── Ppyoloeloss/loss = 1.5149
│       ├── Epoch N-1      = 1.51   (↗ 0.005)
│       └── Best until now = 1.51   (↗ 0.005)
└── Validation
    ├── Ppyoloeloss/loss_cls = 0.7149
    │   ├── Epoch N-1      = 0.6842 (↗ 0.0307)
    │   └── Best until now = 0.6842 (↗ 0.0307)
    ├── Ppyoloeloss/loss_iou = 0.1901
    │   ├── Epoch N-1      = 0.1685 (↗ 0.0216)
    │   └── Best until now = 0.1644 (↗ 0.0257)
    ├── Ppyoloeloss/loss_dfl = 0.8447
    │   ├── Epoch N-1      = 0.7832 (↗ 0.0616)
    │   └── Best until now = 0.7681 (↗ 0.0767)
    ├── Ppyoloeloss/loss = 1.6125
    

Validating epoch 21: 100%|██████████| 10/10 [00:15<00:00,  1.59s/it]


SUMMARY OF EPOCH 21
├── Train
│   ├── Ppyoloeloss/loss_cls = 0.7118
│   │   ├── Epoch N-1      = 0.721  (↘ -0.0091)
│   │   └── Best until now = 0.7191 (↘ -0.0072)
│   ├── Ppyoloeloss/loss_iou = 0.1576
│   │   ├── Epoch N-1      = 0.1601 (↘ -0.0024)
│   │   └── Best until now = 0.1597 (↘ -0.0021)
│   ├── Ppyoloeloss/loss_dfl = 0.7798
│   │   ├── Epoch N-1      = 0.7876 (↘ -0.0078)
│   │   └── Best until now = 0.7825 (↘ -0.0027)
│   └── Ppyoloeloss/loss = 1.4958
│       ├── Epoch N-1      = 1.5149 (↘ -0.0191)
│       └── Best until now = 1.51   (↘ -0.0142)
└── Validation
    ├── Ppyoloeloss/loss_cls = 0.6902
    │   ├── Epoch N-1      = 0.7149 (↘ -0.0247)
    │   └── Best until now = 0.6842 (↗ 0.006)
    ├── Ppyoloeloss/loss_iou = 0.1703
    │   ├── Epoch N-1      = 0.1901 (↘ -0.0198)
    │   └── Best until now = 0.1644 (↗ 0.0059)
    ├── Ppyoloeloss/loss_dfl = 0.7905
    │   ├── Epoch N-1      = 0.8447 (↘ -0.0542)
    │   └── Best until now = 0.7681 (↗ 0.0225)
    ├── Ppyoloeloss/loss 

Train epoch 22:   0%|          | 0/32 [00:00<?, ?it/s]

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /scratch/zw2688/DL_project/yolonas_checkpoints/bball_detect/RUN_20231103_213823_532374

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
